
Our operators are very sparse. Thus, we will encode one operator as a dictionary
of {pauli: coefficient} pairs.
Pauli is an integer. For instance,
XYZ would be encoded as 1 * 4^0 + 2 * 4^1 + 3 * 4^2 = 1 + 8 + 48 = 57.

Observe that we are now basically working base 16. One qublock is two qubits.

XZX really represents
XZX + IIXZX + IIIIXZX + ...

and is thus different from IXZX (but is the same as IIXZX).

Cool. Now, a convolution operation on two Pauli strings will return a dictionary of {pauli : coefficient}

If the first string is a qublocks long and the second is b qublocks long, then the final
strings are at most a + b - 1 qublocks long.

We'll start with the ability to do AB for all A,B in {I,X,Y,Z}^2 and IAB for all A,B in {X,Y,Z}^2.

In [1]:

import numpy as np
import random

n = 3
qubits = 2 * n
num_basis = 4**(qubits)


def reduce(A):
    """
    Divide A by 16 until A is not divisible by 16.
    """
    while A % 16 == 0:
        A //= 16
    return A


def commute(A, B):
    """
    A and B are unreduced pauli strings.
    Return reduced commutator of A,B, in the form {pauli : coefficient}
    """
    res = A ^ B  # XOR
    # Other than res, we also need to keep track of the phase.
    phase = 0  # i^phase is the phase.
    while A > 0 and B > 0:
        # Let a and b be the k-th pauli of i and j respectively.
        a = A % 4
        b = B % 4
        if a > 0 and b > 0 and a != b:  # Else no phase.
            if (b - a) % 3 == 1:  # XY = iZ for instance.
                phase += 1
            elif (b - a) % 3 == 2:  # XZ = -iY for instance.
                phase += 3
            else:
              raise Exception("Bruh")
        A //= 4
        B //= 4
    if phase % 4 == 3:
        return reduce(res), -1
    elif phase % 4 == 1:
        return reduce(res), 1
    else:
        return None, 0


def single_convolve(A, B):
    """
    A and B are single pauli strings

    :param A: pauli string
    :param B: pauli string
    :return: dictionary of {pauli : coefficient}
    """
    res = {}
    for i in range(n):
        shift_B = B * (16 ** i)
        truncate_top = shift_B % (16 ** n)  # Rotate
        truncate_bottom = shift_B // (16 ** n)  # Rotate

        # E.g., ABC00 -> C00 + AB

        key, val = commute(A, truncate_bottom + truncate_top)
        if key is not None:
            if key in res:
                res[key] += val
            else:
                res[key] = val
    return res


def convolve(A, B):
    """
    A and B are dictionaries of {pauli : coefficient}
    """
    res = {}
    for a in A:
        for b in B:
            new = single_convolve(a, b)
            for n in new:
                if n in res:
                    res[n] += A[a] * B[b] * new[n]
                else:
                    res[n] = A[a] * B[b] * new[n]
    # Remove zero coefficients
    remove = []
    for r in res.keys():
        if res[r] == 0:
            remove.append(r)
    for r in remove:
        del res[r]
    return res


def gcd_reduce(A):
    """
    A is a dictionary of {pauli : coefficient}
    """
    gcd = 0
    for a in A:
        gcd = np.gcd(gcd, A[a])
    for a in A:
        A[a] //= gcd
    return A

def invert(A):
    """
    A is a dictionary of {pauli : coefficient}
    """
    res = {}
    for a in A:
        res[a] = -A[a]
    return res

def pretty(A):
    """
    Print a pauli dictionary in a pretty way.
    """
    res = ""
    for a in A:
        if A[a] != 1 and A[a] != -1:
            res += str(A[a])
        elif A[a] == -1:
            res += "-"
        if a == 0:
            res += "I"
        else:
            while a > 0:
                res += "IXYZ"[a % 4]
                a //= 4
        res += " + "
    return res[:-3]


def array(A):
    """
    Return a numpy array of the pauli dictionary
    """
    res = np.zeros(num_basis)
    for a in A:
        res[a] = A[a]
    return res


def cycle(A):
    res = []
    permutations = [(0, 1, 2, 3), (0, 1, 3, 2), (0, 2, 1, 3),
                    (0, 2, 3, 1), (0, 3, 1, 2), (0, 3, 2, 1)]
    for p in permutations:
        new = {}
        for i in A:
            idx = 0
            for j in range(n):  # lmao oof
                # Let a be the j-th pauli of i.
                a = (i // 4**j) % 4
                idx += p[a] * 4**j
            new[reduce(idx)] = A[i]
        res.append(new)

    # Now, try cyclic slides.
    for p in permutations:
        new = {}
        for i in A:
            idx = 0
            for j in range(n):
                # Let a be the j-th pauli of i.
                a = (i // 4**j) % 4
                idx += p[a] * 4**(j + 1)
            new[reduce(idx)] = A[i]
        res.append(new)

    return res



In [ ]:
space = []
iqueue = []

for i in range(16):
    space.append({i: 1})  # AB

iqueue.append(0)  # I lol
iqueue.append(1)  # X
iqueue.append(4)  # IX

for i in range(1, 4):
    for j in range(1, 4):
        space.append({i*4 + j * 16: 1})  # IAB for A,B in {X,Y,Z}

iqueue.append(len(space)-2)  # ZY
# ZZ # These are the only two distinguishable under cycling.
iqueue.append(len(space)-1)

jqueue = iqueue[:] # Copy this; we'll iterate over everything again in the vector context.

# Pretty-print the space
for i,j in enumerate(space):
    print(pretty(j), "*" * (i in iqueue))

In [ ]:
# Save all output in the below in a file, appending
f = open("tier_output.txt", "a")

def fprint(*args):
    print(*args)
    f.write(*args)


# while len(iqueue) > 0:
#     i = iqueue.pop(0)
#     fprint(" ------------------ ")
#     fprint(i, " => ", pretty(space[i]))
#     for j in range(i):
#         res = convolve(space[i], space[j])
#         if len(res) != 1:
#             continue
#         res = gcd_reduce(res)
        
#         cycle_space = cycle(res)
#         # Add all the new elements to space, checking rank
#         added = False
#         for c in cycle_space:
#             if c not in space and invert(c) not in space:
#                 space.append(c)
#                 added = True
#         if not added:
#             continue
#         iqueue.append(len(space)-1)
#         jqueue.append(len(space)-1)

#         fprint(pretty(res), " = (", pretty(
#             space[i]), " * ", pretty(space[j]), ")")
#     f.flush()
# f.close()

We no longer restrict ourselves to pure operators.

In [ ]:
vector_space = []
for i, s in enumerate(space):
    vector_space.append(array(s))

# iqueue = jqueue[:]

def singular(M):
    mat = np.array(M)
    rank = np.linalg.matrix_rank(mat)
    return rank < len(M)

while len(iqueue) > 0:
    assert len(vector_space) == len(space)
    i = iqueue.pop(0)
    fprint(" ------------------ ")
    fprint(i, " => ", pretty(space[i]))
    for j in range(i):
        res = convolve(space[i], space[j])
        res = gcd_reduce(res)

        cycle_space = cycle(res)
        # Add all the new elements to space, checking rank
        added = False
        for c in cycle_space:
            if singular(vector_space + [array(c)]):
                continue
            vector_space.append(array(c))
            space.append(c)
            added = True
        if not added:
            continue

        iqueue.append(len(space)-1)

        fprint(pretty(res), " = (", pretty(
            space[i]), " * ", pretty(space[j]), ")")
    f.flush()
f.close()